Рассмотрим вирус Гепатита С, а точнее геном [NC_030791.1](https://www.ncbi.nlm.nih.gov/nuccore/NC_030791)

Требуется посчитать количество Z-DNA и квадруплексов, а также указать, куда они попадают

###1

In [ ]:
!chmod a+x zhunt2
!./zhunt2 12 8 12 sequence.fasta
!cat sequence.fasta.Z-SCORE

Рассмотрим данные, используя read_csv

In [6]:
import pandas as pd
z=pd.read_csv("sequence.fasta.Z-SCORE", skiprows=1, names=["Start","End","1","2","3","Score","Seq","4"], delim_whitespace=True)

Проверим наличие строк с Z-score более 300

In [7]:
z.loc[z['Score'] >= 300]

,Start,End,1,2,3,Score,Seq,4
230,231,255,24,21.894,49.817,461.6513,cggagatttgggcgtgcccccgcg,ASASASASASASASASASASASAS
231,232,254,22,22.236,40.826,313.1476,ggagatttgggcgtgcccccgc,SASASASASASASASASASASA
232,233,255,22,21.747,51.301,548.0334,gagatttgggcgtgcccccgcg,ASASASASASASASASASASAS
233,234,254,20,22.036,43.138,392.3850,agatttgggcgtgcccccgc,SASASASASASASASASASA
234,235,255,20,21.605,52.028,648.0815,gatttgggcgtgcccccgcg,ASASASASASASASASASAS
...,...,...,...,...,...,...,...,...
7172,7173,7189,16,21.061,40.638,1263.2160,gtctgcgtacgcgtaa,SASASASASASASASA
7173,7174,7190,16,21.439,39.807,791.4059,tctgcgtacgcgtaag,ASASASASASASASAS
7174,7175,7191,16,21.283,37.193,958.1629,ctgcgtacgcgtaagg,SASASASASASASASA
7175,7176,7192,16,21.539,38.918,701.2976,tgcgtacgcgtaaggt,ASASASASASASSASA


Это все участки Z-DNA, в данном гене их 117

### 2

Импортируем библиотеки, требуемые для оценки числа квадруплексов

In [8]:
!pip install biopython
import re
from Bio import SeqIO

     |████████████████████████████████| 2.3MB 4.4MB/s 


Посчитаем квадруплексы, заметим что цифра "3" в следующем коде означает минимум вхождений нуклеотидных последовательностей.

In [9]:
input_file = "sequence.fasta"
fasta_sequence = SeqIO.parse(input_file,'fasta')
for record in SeqIO.parse(input_file, "fasta"):
  print("%s %i" % (record.id, len(record)))
  name, sequence = record.id, str(record.seq)
pattern="(?:G{3,}[ATGC]{1,7}){3,}G{3,}"
PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern,sequence)]
len(PQS)

NC_030791.1 9443


0

Заметим, что при "3" вхождений не нашлось, проверим "2", так как это минимальное значение, при нуле которого, квадруплексов точно нет

In [10]:
input_file = "sequence.fasta"
fasta_sequence = SeqIO.parse(input_file,'fasta')
for record in SeqIO.parse(input_file, "fasta"):
  print("%s %i" % (record.id, len(record)))
  name, sequence = record.id, str(record.seq)
pattern="(?:G{2,}[ATGC]{1,7}){2,}G{2,}"
PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern,sequence)]
len(PQS)

NC_030791.1 9443


69

Код показал, что есть 69 квадруплексов, рассмотрим их позицию в коде и саму оследовательность

In [ ]:
PQS

###3

Определим в какие участки генома попадают Z-ДНК и квадруплексы
Для этого зайдем на страницу вируса в ncbi и рассмотрим features. Скопируем их в текстовый файл, названный "gene.txt". Пройдемся по полученному файлу функцией grep, чтобы выделить только координаты генов (находятся в файле после gene)

In [14]:
!cat gene.txt | grep "gene\s" > LocGene.txt
!cat LocGene.txt

     gene            340..9381


In [13]:
!cat gene.txt | grep "mat_peptide\s" > LocPept.txt
!cat LocPept.txt

     mat_peptide     340..912
     mat_peptide     913..1488
     mat_peptide     1489..2589
     mat_peptide     2590..2778
     mat_peptide     2779..3429
     mat_peptide     3430..5322
     mat_peptide     5323..5484
     mat_peptide     5485..6267
     mat_peptide     6268..7605
     mat_peptide     7606..9378


Осталось заметить, что как Z-DNA, так и квадруплексы попадают как в гены, так и в межгенное пространство